In [3]:
from dotenv import load_dotenv
load_dotenv()

True

### 빠른 시작 샘플 코드

In [6]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from pydantic import BaseModel

class Weather(BaseModel):
    temperature: float
    condition: str

def weather_tool(city: str) -> str:
    """Get weather for a given city."""
    return f"it's sunny and 70 degrees in {city}"

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[weather_tool],
    response_format=Weather
)

result = agent.invoke({"messages": [HumanMessage("What`s the weather in SF?")]})
print(repr(result["structured_response"]))

Weather(temperature=70.0, condition='sunny')


### 시스템 프롬프트 정의
    구체적이고 실행 가능한 방식으로 작성

In [8]:
system_prompt = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. 
If you can tell from the question that they mean whereever they are, use the get_user_location tool to find their location."""

### 도구 생성

In [ ]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

def get_weather_for_location(city: str) -> str:  # (1)!
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

from langchain_core.runnables import RunnableConfig

USER_LOCATION = {
    "1": "Florida",
    "2": "SF"
}

@tool
def get_user_location(config: RunnableConfig) -> str:
    """Retrieve user information based on user ID."""
    runtime = get_runtime()
    user_id = runtime.context['user_id']
    return USER_LOCATION[user_id]

In [13]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="openai:gpt-4o-mini",
    temperature=0
)

### 에이전트가 예측 가능한 형식으로 데이터를 반환하도록 보장 `DataClass`

In [3]:
from dataclasses import dataclass

@dataclass
class WeatherResponse:
    conditions: str
    punny_response: str

### 대화 내역을 기억하도록 설정

In [18]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

In [19]:
agent = create_agent(
    model=model,
    prompt=system_prompt,
    tools=[get_user_location, get_weather_for_location],
    response_format=WeatherResponse,
    checkpointer=checkpointer
)

config = {"configurable": {"thread_id": "1"}}
context = {"user_id": "1"}
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=context
)

response['structured_response']

response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=context
)

response['structured_response']

Runtime(context={'user_id': '1'}, store=None, stream_writer=<function Pregel.stream.<locals>.stream_writer at 0x00000292EFC637E0>, previous=None)


WeatherResponse(conditions='sunny', punny_response="You're welcome! I'm just trying to brighten your day!")